In [1]:
import torch
import torchvision
import torchvision.transforms as transforms

## Load dataset and create dataloaders

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=1000,
                                          shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1000,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [3]:
# 3 channels (rbg), where each row contains 32x32 colour image for the current colour
#next(iter(test_loader))[0][0].shape

## Network architecture

In [4]:
from torch.nn import BatchNorm2d
from torch import nn

class ConvNet(nn.Module):
    def __init__(self, dropout_rate, activation_function_type):
        super(ConvNet, self).__init__()

        if activation_function_type == "relu":
            activation_function = nn.ReLU()
        elif activation_function_type == "sigmoid":
            activation_function = nn.Sigmoid()
        elif activation_function_type == "leaky_relu":
            activation_function = nn.LeakyReLU()

        self.layer1 = nn.Sequential(
            # 3 is the rbg
            nn.Conv2d(3, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            activation_function,
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.layer2 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(64),
            activation_function,
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.layer3 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(128),
            activation_function,
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.drop_out = nn.Dropout(p=dropout_rate)
        self.fc1 = nn.Linear(4 * 4 * 128, 1000)
        self.fc2 = nn.Linear(1000, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc2(out)
        return out

Training 27 variations of the network with different parameters

In [5]:
from utils import train
import torch.optim as optim

learning_rate = 0.01
epochs = 100
criterion = nn.CrossEntropyLoss()

list_of_dropout_rates = [0, 0.25, 0.5]
list_of_activation_functions = ['sigmoid', 'relu', 'leaky_relu']
list_of_optimizers = ['SGD', 'SGD_momentum', 'Adam']

for dropout_rate in list_of_dropout_rates:
    for activation_function_type in list_of_activation_functions:
        for optimizer_type in list_of_optimizers:
            net = ConvNet(dropout_rate=dropout_rate, activation_function_type=activation_function_type)
            if optimizer_type == 'Adam':
                optimizer = optim.Adam(net.parameters(), lr=learning_rate)
            elif optimizer_type == 'SGD':
                optimizer = optim.SGD(net.parameters(), lr=learning_rate)
            elif optimizer_type == 'SGD_momentum':
                 optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)

            train(epochs=epochs, model=net, optimizer=optimizer, 
                criterion=criterion, train_loader=train_loader, 
                test_loader=test_loader, 
                file_name='dropout_{}_actvfunc_{}_optimizer_{}'.format(
                    dropout_rate, activation_function_type, optimizer_type))

Loss: 0.413 |  Test Loss: 0.672 | Train acc: 85.358 | Test acc: 78.24 | Time taken: 0:00:17.517927
Epoch: 30 | Train Loss: 0.403 |  Test Loss: 0.756 | Train acc: 85.58 | Test acc: 75.8 | Time taken: 0:00:15.771782
Epoch: 31 | Train Loss: 0.398 |  Test Loss: 0.746 | Train acc: 85.96 | Test acc: 75.58 | Time taken: 0:00:14.537924
Epoch: 32 | Train Loss: 0.38 |  Test Loss: 0.626 | Train acc: 86.478 | Test acc: 79.08 | Time taken: 0:00:14.682833
Epoch: 33 | Train Loss: 0.368 |  Test Loss: 0.675 | Train acc: 86.96 | Test acc: 77.59 | Time taken: 0:00:14.494446
Epoch: 34 | Train Loss: 0.367 |  Test Loss: 0.668 | Train acc: 86.974 | Test acc: 78.03 | Time taken: 0:00:14.438212
Epoch: 35 | Train Loss: 0.357 |  Test Loss: 0.725 | Train acc: 87.22 | Test acc: 76.17 | Time taken: 0:00:14.429940
Epoch: 36 | Train Loss: 0.362 |  Test Loss: 0.704 | Train acc: 86.91 | Test acc: 78.08 | Time taken: 0:00:14.393190
Epoch: 37 | Train Loss: 0.342 |  Test Loss: 0.633 | Train acc: 87.754 | Test acc: 79.91 |

## Create the optimal model with data augmentation

In [8]:
import torch
import torchvision
import torchvision.transforms as transforms
import PIL

## transform link - https://colab.research.google.com/drive/109vu3F1LTzD1gdVV6cho9fKGx7lzbFll#scrollTo=yLEwF_2RzGs0

## While iterating the train_loader, we will get random transformations on the fly
## Each epoch they will be different basically
train_transform = transforms.Compose([
     torchvision.transforms.ColorJitter(hue=.05, saturation=.05),
     torchvision.transforms.RandomHorizontalFlip(),
     torchvision.transforms.RandomRotation(20, resample=PIL.Image.BILINEAR),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
    
test_trainsform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

train_set = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=train_transform)
train_loader = torch.utils.data.DataLoader(train_set, batch_size=1000,
                                          shuffle=True, num_workers=2)

test_set = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=test_trainsform)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=1000,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [9]:
from utils import train
import torch.optim as optim
from torch import nn

learning_rate = 0.01
epochs = 100
criterion = nn.CrossEntropyLoss()

## Optimal network params
optimal_dropout = 0.5
optimal_activation_func_type = 'leaky_relu'
optimal_net = ConvNet(dropout_rate=optimal_dropout, activation_function_type=optimal_activation_func_type)
optimal_optimizer_type = 'SGD_momentum'
optimal_optimizer = optim.SGD(optimal_net.parameters(), lr=learning_rate, momentum=0.9)

train(epochs=epochs, model=optimal_net, optimizer=optimal_optimizer, 
                criterion=criterion, train_loader=train_loader, 
                test_loader=test_loader, 
                file_name='dropout_{}_actvfunc_{}_optimizer_{}_augmented'.format(
                    optimal_dropout, optimal_activation_func_type, optimal_optimizer_type))

Epoch: 1 | Train Loss: 1.765 |  Test Loss: 1.452 | Train acc: 35.884 | Test acc: 47.88 | Time taken: 0:00:19.774976
Epoch: 2 | Train Loss: 1.424 |  Test Loss: 1.272 | Train acc: 48.332 | Test acc: 54.21 | Time taken: 0:00:19.665445
Epoch: 3 | Train Loss: 1.308 |  Test Loss: 1.136 | Train acc: 52.722 | Test acc: 59.44 | Time taken: 0:00:20.288205
Epoch: 4 | Train Loss: 1.205 |  Test Loss: 1.107 | Train acc: 56.94 | Test acc: 61.03 | Time taken: 0:00:23.053602
Epoch: 5 | Train Loss: 1.143 |  Test Loss: 1.018 | Train acc: 59.142 | Test acc: 64.15 | Time taken: 0:00:34.271989
Epoch: 6 | Train Loss: 1.091 |  Test Loss: 0.993 | Train acc: 61.142 | Test acc: 64.37 | Time taken: 0:00:33.246393
Epoch: 7 | Train Loss: 1.046 |  Test Loss: 0.96 | Train acc: 63.102 | Test acc: 67.13 | Time taken: 0:00:33.887815
Epoch: 8 | Train Loss: 1.018 |  Test Loss: 0.94 | Train acc: 64.064 | Test acc: 67.64 | Time taken: 0:00:33.890766
Epoch: 9 | Train Loss: 0.983 |  Test Loss: 0.89 | Train acc: 65.332 | Test 